In [ ]:
# Импортируем необходимые библиотеки и модули

In [1]:
import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import glob

from sklearn.model_selection import train_test_split

import random
import shutil
import os

In [ ]:
# Определяем устройство для выполнения вычислений

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

seed = 42 #Устанавливаем случайное зерно для генератора псевдослучайных чисел для воспроизводимости результатов
torch.manual_seed(seed)
if device == "cuda":
    torch.cuda.manual_seed(seed)

np.random.seed(seed)

cpu


In [ ]:
# Определяем класс Dataset для загрузки и предобработки изображений

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
    # Определяем методы len и getitem для доступа к данным в Dataset
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = os.path.split(os.path.split(img_path)[-2])[-1]
        if label == 'Dog':
            label = torch.tensor(1)
        elif label == 'Cat':
            label = torch.tensor(0)

        return img_transformed, label

In [ ]:
# Создаем объект преобразований для предобработки изображений 

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

In [ ]:
# Определяем путь к директории с изображениями

In [8]:
img_dir = 'PetImages'

img_list = glob.glob(os.path.join(img_dir, '*/*.jpg')) # Создаем список файлов изображений с помощью функции glob

In [ ]:
# Разделяем список файлов на обучающую и валидационную выборки с помощью функции train_test_split

In [9]:
train_list, val_list = train_test_split(img_list, test_size=0.2)

In [ ]:
# Создаем объекты Dataset для обучающей и валидационной выборок

In [10]:
train_data = Dataset(train_list, transform=transform)
val_data = Dataset(val_list, transform=transform)

In [ ]:
# Создаем загрузчики данных для обучающей и валидационной выборок

In [11]:
batch_size = 32

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [ ]:
# Определяем класс модели сверточной нейронной сети с несколькими сверточными слоями и полносвязными слоями

In [12]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)

        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)

        )

        self.fc1 = nn.Linear(3 * 3 * 64, 10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10, 2)
        self.relu = nn.ReLU()

    def forward(self, x): #Определяем метод forward, который определяет поток данных внутри модели
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)

        return out

In [ ]:
# Создаем экземпляр модели и переносим его на устройство

In [13]:
model = Cnn().to(device)
model.train()

Cnn(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=576, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
  (re

In [ ]:
# Определяем оптимизатор и функцию потерь

In [14]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [19]:
epochs = 10

for epoch in range(epochs): #Внутри цикла обучения выполняются итерации по обучающему загрузчику данных
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in train_loader: # Переносим данные и метки на устройство
        data = data.to(device)
        label = label.to(device)

        output = model(data) #Получаем выход модели и вычисляем потерю
        loss = criterion(output, label)

        optimizer.zero_grad() # Обнуляем градиенты, выполняем обратное распространение ошибки и обновляем веса модели
        loss.backward()
        optimizer.step()

        acc = ((output.argmax(dim=1) == label).float().mean()) #Вычисляем точность и потерю на обучающей выборке
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch + 1, epoch_accuracy, epoch_loss)) # Выводимрезультаты обучения на каждой эпохе
# Вне цикла обучения выполняем валидацию модели на валидационном загрузчике данных
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in val_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = ((val_output.argmax(dim=1) == label).float().mean())
            epoch_val_accuracy += acc / len(val_loader)
            epoch_val_loss += val_loss / len(val_loader)

        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch + 1, epoch_val_accuracy, epoch_val_loss))

Epoch : 1, train accuracy : 0.8273997902870178, train loss : 0.38077840209007263
Epoch : 1, val_accuracy : 0.8109079599380493, val_loss : 0.41418910026550293
Epoch : 2, train accuracy : 0.851849377155304, train loss : 0.33566999435424805
Epoch : 2, val_accuracy : 0.8144906759262085, val_loss : 0.4108200967311859
Epoch : 3, train accuracy : 0.875150203704834, train loss : 0.2944703996181488
Epoch : 3, val_accuracy : 0.8258362412452698, val_loss : 0.39464524388313293
Epoch : 4, train accuracy : 0.8916997909545898, train loss : 0.2559053897857666
Epoch : 4, val_accuracy : 0.8055335879325867, val_loss : 0.4616509974002838
Epoch : 5, train accuracy : 0.9083508849143982, train loss : 0.22029808163642883
Epoch : 5, val_accuracy : 0.8202627301216125, val_loss : 0.4712638258934021
Epoch : 6, train accuracy : 0.9216514229774475, train loss : 0.19403323531150818
Epoch : 6, val_accuracy : 0.8138935565948486, val_loss : 0.4873391091823578
Epoch : 7, train accuracy : 0.9314520359039307, train loss :